## Embedding Synthesis Demo for GloVe

To make sure that you have everything needed, we will start from scratch for this notebook.  \
There is only one part that is not included (gathering wikitext -- or any other corpus) -- we assume it has been collected and is sitting on harddisk\
It is advised to run this notebook in a virtual environment \
Once you have done that, let's get the dependencies!

In [ ]:
!pip3 install torch tqdm wandb nltk selenium beautifulsoup4

_**Note**_: section 5 uses selenium web crawler and firefox, so it would be good if the browser is installed. Otherwise you would have to get the respective browser driver yourself (for e.g for chrome user it's [ChromeDriver](https://chromedriver.chromium.org/home)) and then modify the [function](#driver) here.

There are five sections to this notebook:
* [Section 1: Preparing Corpus](#s1)
* [Section 2: Preparing Training Data](#s2)
* [Section 3: Training with Pytorch and Wandb](#s3)
* [Section 4: Testing](#s4)
* [Section 5: Inference](#s5)
* [Section 6: Incorporate Synthetic Vector Into GloVe](#s6)

## <a class="anchor" id="s1">Section 1: Preparing Corpus</a>

In [2]:
from tqdm import tqdm
import requests



url = 'https://nlp.stanford.edu/data/glove.6B.zip'

dest_file = './glove_6B.zip'
# Streaming, so we can iterate over the response.
response = requests.get(url, stream=True)
total_size_in_bytes= int(response.headers.get('content-length', 0))
block_size = 1024 #1 Kibibyte
progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True)
with open(dest_file, 'wb') as file:
    for data in response.iter_content(block_size):
        progress_bar.update(len(data))
        file.write(data)
progress_bar.close()
if total_size_in_bytes != 0 and progress_bar.n != total_size_in_bytes:
    print("ERROR, something went wrong")

100%|██████████████████████████████████████████████████████████████████████████████| 862M/862M [03:39<00:00, 3.93MiB/s]


In [6]:
#extract to folder
import zipfile
with zipfile.ZipFile(dest_file, 'r') as zip_ref:
    zip_ref.extractall(dest_file.replace('.zip','') )  #use the filename as destination dir


In [9]:
#generate the glove files
import argparse
import numpy as np
import sys
import json

def generate(file):
    words = []
    vectors = {}
    with open(file, 'r',encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines:
            _temp = line.rstrip().split(' ')
            words.append(_temp[0])
            vectors[_temp[0]] = [float(x) for x in _temp[1:]]

    vocab_size = len(words)
    vocab = {w: idx for idx, w in enumerate(words)}
    ivocab = {idx: w for idx, w in enumerate(words)}

    vector_dim = len(vectors[ivocab[0]])
    W = np.zeros((vocab_size, vector_dim))
    for word, v in vectors.items():
        if word == '<unk>':
            continue
        W[vocab[word], :] = v

    # normalize each word vector to unit variance
    W_norm = np.zeros(W.shape)
    d = (np.sum(W ** 2, 1) ** (0.5))
    W_norm = (W.T / d).T
    return (W_norm, vocab, ivocab)

glove_file = f"{dest_file.replace('.zip','')}/glove.6B.100d.txt"
(W_norm, vocab, ivocab) = generate(glove_file)

#save the files as npy for easier loading 
np.save('./glove6B100d.npy',W_norm)
with open('./glove_vocab.json','w') as f:
    json.dump(vocab,f)
with open('./glove_ivocab.json','w') as f:
    json.dump(ivocab,f)

## <a class="anchor" id="s2">Section 2: Preparing Training Data</a>

In [12]:
#this section covers the preparation pipeline. It is assumed that the wikitext is already downloaded and extracted
# the wikipedia dump is here https://dumps.wikimedia.org/enwiki/20210920/enwiki-20210920-pages-articles-multistream.xml.bz2 
# the source code to extract it is here https://github.com/attardi/wikiextractor 


#eventually, this part of the code just need the path to all the text files, 
#so it's up to you to implement it how you like
import os, re
wiki_dataset_dir = "D:/DATASETS_UNZIP/Datasets/text"
#Extracting from wiki 
filepath_list = []
wiki=True
if wiki == True:
    for folder in os.listdir(wiki_dataset_dir):
        for file in os.listdir(wiki_dataset_dir+'/'+folder):
            filepath_list.append(wiki_dataset_dir+'/'+folder+'/'+file)

In [ ]:
from nltk.corpus import stopwords

In [ ]:
#run this if you dont have nltk stopwords
nltk.download('stopwords')

In [ ]:
stopword_list = stopwords.words('english')

In [14]:
#extract the target word and neighboring word from its sentences

def extract(sentence,target_word,context_length,pad,debug=True):
    target_word = target_word.lower()
    sentence = sentence.lower()
    if target_word not in sentence: #reduce processing
        return
    
    s = sentence.lower().strip()
    s = re.sub('[\n\r\ ]+',' ',s)
    s = re.sub('[^a-z ]+','',s)
    
    t = target_word
    raw_tokens = s.split(' ')
    tokens = [i for i in raw_tokens if i != '' and i not in stopword_list]
    word_list = []
    if t in tokens:
        __index = tokens.index(t)        # this only get one utterance, what about other utterances? 
        if __index < context_length:      #pad front
            word_list += [pad for _ in range(context_length-__index)]
            word_list += tokens[:__index]      #pad back
        else:
            word_list += tokens[__index-context_length:__index]
        if __index + context_length >= len(tokens):
            word_list += tokens[__index+1:]
            word_list += [pad for _ in range(context_length + __index + 1 - len(tokens))]
        else:
            word_list += tokens[__index+1:__index+context_length+1]
        return word_list


    else:   #target not found:
        return None


In [17]:
!mkdir processed_data

In [ ]:
#crawling multiple words at once, writing the file into target directory  

target_words = ['tired','pointless']
                
context_length = 10

for target_word in target_words:
    line_written = 0
    corpus = f"./processed_data/{target_word}_corpus_c{context_length}.txt"
    with open(corpus,'a') as o10:
        for file in tqdm(filepath_list):
            with open(file,'r',encoding='utf-8') as f:
                lines = f.readlines()
                for line in lines:
                    result = extract(line,target_word,context_length,"<pad>",True)
                    if result == None:
                        continue
                    else:
                        o10.write(f'{target_word}:')
                        o10.write(''.join([i+' ' for i in result] ))
                        o10.write('\n')
                        line_written += 1
                        if line_written % 10000 == 0:
                            print(f"Written {line_written} into file {target_word}")
        print("\n\n DONE WITH CORPUS {corpus}\n\n")

## <a class="anchor" id="s3">Section 3: Training with Pytorch and Wandb</a>

In [23]:
import json
import numpy as np
import wandb
import random
from datetime import date

wandb.init(project='Synthetic Net')
config = wandb.config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sosig_catto (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [24]:
import torch
import torch.nn as nn

class DenseNet(nn.Module):
    def __init__(self,context_length,embed_size=100):
        super().__init__()
        self.n = context_length*2
        self.embed_size = 100
        self.act = nn.ReLU()
        self.out = nn.Tanh() 
        self.hidden1 = nn.Linear(self.n*self.embed_size,2048)
        self.hidden2 = nn.Linear(2048,512)
        self.hidden3 = nn.Linear(512,self.embed_size)
 
    def forward(self,x):
        x = x.view(x.size(0), -1)
        x = self.act(self.hidden1(x))
        x = self.act(self.hidden2(x))
        x = self.out(self.hidden3(x))
        return x
    
config.context_length = 10
model = DenseNet(context_length = config.context_length)
print(model)

DenseNet(
  (act): ReLU()
  (out): Tanh()
  (hidden1): Linear(in_features=2000, out_features=2048, bias=True)
  (hidden2): Linear(in_features=2048, out_features=512, bias=True)
  (hidden3): Linear(in_features=512, out_features=100, bias=True)
)


In [25]:
#util file contains the loading functions
from util import *

#Loading the data
W_norm,vocab,ivocab = load_glove(        
        weight_file = './glove6B100d.npy',
        vocab_file = './glove_vocab.json',
        ivocab_file='./glove_ivocab.json'
)
    
config.batch_size = 64

#configure the files used for training. Can load multiple files
#usually load with negative samples to avoid overfitting
files_for_training =['tired'] 

training_files = [f'./processed_data/{x}_corpus_c10.txt' for x in files_for_training]

training_data = load_training_batch(training_files,config.batch_size)

#for logging purpose, provide data lineage
config.data = "wiki_only"

train_tensor = get_embedding(training_data,W_norm,vocab)

In [177]:
### checking vocab
print(len(vocab))
print(len(ivocab))

400000
400000


In [26]:
import torch.optim as optim


config.lr = 0.0005
config.momentum = 0.005
optimizer = optim.SGD(model.parameters(),lr=config.lr,momentum=config.momentum,weight_decay=0.01)
criterion = nn.L1Loss()

def cosim(v1,v2):
    return np.dot(v1,v2)/(np.linalg.norm(v1)*np.linalg.norm(v2))

#scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, debug_set.shape[0], eta_min=config.lr)
#learning rate adjustment -- try 0.001

def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    for batch in iterator:
        optimizer.zero_grad()
        features,labels = batch
        batch_size = features.shape[0]
        predictions = model(torch.Tensor(features)).squeeze(1)
        loss = criterion(predictions,torch.Tensor(labels))      
        loss.backward()
        
        optimizer.step()
        epoch_loss += loss.item()
        
        cosim_score = np.mean([cosim(labels[i],predictions[i].detach().numpy()) for i in range(batch_size) ])
        
    return epoch_loss,cosim_score

In [36]:
config.epochs = 40   #usually 40 is the best

best_valid_loss = float('inf')

for epoch in tqdm(range(config.epochs)):   
    train_loss,cosim_score= train(model,iter(train_tensor), optimizer, criterion)

    #epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    #if valid_loss < best_valid_loss:
     #   best_valid_loss = valid_loss
      #  torch.save(model.state_dict(), 'tut1-model.pt')
    
    #print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    wandb.log({"loss":train_loss,"cosim_score":cosim_score})
    print(f'Epoch:{epoch+1:02}\t|\tTrain Loss: {train_loss:.3f}\t|\tCosim score: {cosim_score:.3f}')

    

  2%|██                                                                                 | 1/40 [00:07<04:48,  7.39s/it]

Epoch:01	|	Train Loss: 18.281	|	Cosim score: 0.139


  5%|████▏                                                                              | 2/40 [00:14<04:43,  7.46s/it]

Epoch:02	|	Train Loss: 17.854	|	Cosim score: 0.194


  8%|██████▏                                                                            | 3/40 [00:22<04:37,  7.49s/it]

...

Epoch:38	|	Train Loss: 7.019	|	Cosim score: 0.910


 98%|███████████████████████████████████████████████████████████████████████████████▉  | 39/40 [05:21<00:08,  8.14s/it]

Epoch:39	|	Train Loss: 6.844	|	Cosim score: 0.914


100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [05:29<00:00,  8.24s/it]

Epoch:40	|	Train Loss: 6.676	|	Cosim score: 0.918


In [28]:
! mkdir output
torch.save(model.state_dict(),f'output/{date.today().strftime("%Y-%m")}_{config.data}_{wandb.run.name}.pt')

## <a class="anchor" id="s4">Section 4: Testing</a>

In [33]:
## section 4: Testing using analogy tests
model_to_test =  DenseNet(context_length = 10)
model_to_test.load_state_dict(torch.load('./output/2021-09_wiki_only_cool-hill-39.pt'))

##Testing has 3 unit tests
#Test 1 sentence
#Test 1 batch
#custom sentence


#Test 1 -- random sentence in training
random_sent = random.choice(training_data)[random.randint(0,config.batch_size-1)]
y,x = random_sent.split(':')
x = re.sub('[\n\r\ ]+',' ',x).strip()
sample_tensor = torch.Tensor([[get_glove_vec(word,W_norm,vocab) for word in x.split(' ')]])
sample_output = model_to_test(sample_tensor)
target_label = np.array(get_glove_vec(y,W_norm,vocab))

output1 = sample_output.squeeze(1)
vec_output1 = output.detach().numpy()
print(vec_output1.shape)

def __distance(W, vocab, ivocab, vec_output):


    dist = np.dot(W, vec_output.T).squeeze(1)
    print(dist.shape)
    a = np.argsort(-dist)[:10]

    print("\n                               Word       Unnormalized Cosine distance\n")
    print("---------------------------------------------------------\n")
    for i,x in enumerate(a):
        print("%d%35s\t\t%f" % (i,ivocab[str(x)], dist[x]))
print(f"Test 1 -- sample sentence: \n\n{random_sent}\n\n")

__distance(W_norm,vocab,ivocab,vec_output1)


print(f"\n\n\t\tCosim score: {cosim(vec_output1,target_label)}")


(1, 100)
Test 1 -- sample sentence: 

tired:short run run sweet road runner real scored music ten feathered clippety clobbered used set generic musical cues follow action 



(400000,)

                               Word       Unnormalized Cosine distance

---------------------------------------------------------

0                    multilateralism		0.190550
1                          resurging		0.161833
2                        replicators		0.156357
3                       undiminished		0.147284
4                            sinning		0.145458
5                            finning		0.143951
6                             glatch		0.143233
7                      neoliberalism		0.142868
8                      americanizing		0.142485
9                          labarbera		0.142431


		Cosim score: [0.09054023]


In [85]:
#test 2: test by batch

random_batch = random.choice(training_data)
sample_batch_tensor = []
target_batch_tensor = []
for sentence in random_batch:
    y,x = sentence.split(':')
    x = re.sub('[\n\r\ ]+',' ',x).strip()
    sample_tensor = [get_glove_vec(word,W_norm,vocab) for word in x.split(' ')]
    target_batch_tensor.append(get_glove_vec(y,W_norm,vocab))
    sample_batch_tensor.append(sample_tensor)
    
sample_batch_tensor = torch.Tensor(np.array(sample_batch_tensor))
target_batch_tensor = np.array(target_batch_tensor)

sample_output = model(sample_batch_tensor)

output2 = torch.mean(sample_output,0)   #sum across embeddings
vec_output2 = output2.detach().numpy().reshape((1,100))

print(f"Test 2 -- sample batch: \n\n")

__distance(W_norm,vocab,ivocab,vec_output2)


print(f"\n\n\t\tCosim score: {cosim(vec_output2,target_label)}")

Test 2 -- sample batch: 


(400000,)

                               Word       Unnormalized Cosine distance

---------------------------------------------------------

0                              tired		0.596813
1                             scared		0.462481
2                              weary		0.460485
3                                 'm		0.446562
4                         frustrated		0.442676
5                           fatigued		0.438139
6                          exhausted		0.436740
7                              bored		0.434672
8                             afraid		0.433339
9                               feel		0.431909


		Cosim score: [0.14825795]


In [35]:
###Test 3: Custom


random_sent = 'pacific disaster response fund support armenian government fight spread covid year bank committed million loan electric networks armenia ensure electricity '
target_word = 'pneumonia'
target_label = np.array(get_glove_vec(target_word,W_norm,vocab))
random_sent = re.sub('[\n\r\ ]+',' ',random_sent).strip()

sample_tensor = torch.Tensor([[get_glove_vec(word,W_norm,vocab) for word in random_sent.split(' ')]])
sample_output = model(sample_tensor)
output3 = sample_output.squeeze(1)
vec_output3 = output3.detach().numpy()
print(f"Test 3: Custom Test\n\n{random_sent}\n\n")
__distance(W_norm,vocab,ivocab,vec_output3)
print(f"\n\n\t\tCosim score: {cosim(vec_output3,target_label)}")


Test 3: Custom Test

pacific disaster response fund support armenian government fight spread covid year bank committed million loan electric networks armenia ensure electricity


(400000,)

                               Word       Unnormalized Cosine distance

---------------------------------------------------------

0                    multilateralism		0.190550
1                          resurging		0.161833
2                        replicators		0.156357
3                       undiminished		0.147284
4                            sinning		0.145458
5                            finning		0.143951
6                             glatch		0.143233
7                      neoliberalism		0.142868
8                      americanizing		0.142485
9                          labarbera		0.142431


		Cosim score: [0.12973318]


## <a class="anchor" id="s5">Section 5: Inference</a>

**Note**: the driver function is <a class="anchor" id="driver">get_google_search_page</a>. The driver is currently defined as Firefox

In [93]:
## section 5: generating for an unknown word 

#we will crawl some data for an unknown words using selenium, and export it to a text file

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pprint
import time
import argparse
from nltk.tokenize import sent_tokenize
import re
import requests
from bs4 import BeautifulSoup


def get_google_search_page(input_text):
    fox = webdriver.Firefox()
    fox.get(f"https://www.google.com/search?q={input_text}")
    search_bar = fox.find_element_by_tag_name("input")
    time.sleep(2)
    cur_url = fox.current_url
    fox.close()
    fox.quit()
    return cur_url

def get_top_results(URL):
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')

    all_urls = []
    for elems in soup.findAll('h3'):
        for link in soup.findAll('a'):
            url = str(link.get('href'))
            if url.startswith('/url?'):
                all_urls.append(url[7:])
    return all_urls

def filter_duplicate(url_list,max_url=5):
    domain_list = set()
    result = dict()
    blacklist = ["youtube.com"]
    for url in url_list:
        domain = re.findall(r'^https://www.(.*?)/',url) 
        url = re.findall(r'^https://(.*?)&sa',url) # &sa is there for the google rubbish
        if domain != [] and domain[0] not in domain_list and domain[0] not in blacklist:   #it will not check[0] until domain != [] satisfy
            domain_list.add('https://www.'+domain[0])
            result['www.'+domain[0]] = "https://"+url[0]
            if len(domain_list) == max_url:
                break
    return result



def extract_text_elems_from_urls(urls):
    stored_text = dict()
    for url_ in urls.values():
        #print(f'========{url_}===========')
        try: 
            childpage = requests.get(url_)
            childsoup = BeautifulSoup(childpage.content, 'html.parser')
            stored_text[url_] = {'h1':set(),'p':set()}
            for tagment in ['h1','p']:
                    elements = childsoup.findAll(tagment)
                    for element in elements:
                        stored_text[url_][tagment].add(element.text)
        except:
            print(f"ERROR at url {url_}")
            pass
        #print("========================================================")
    return stored_text


def clean_sentence(sent):
    sent = re.sub(f'[\ \r\n]+',' ',sent)
    return sent.lower()


def write_text_dict_to_file(stored_text,outputfile='test.txt'):
    with open(outputfile,'w',encoding='utf-8') as f:
        for url,texts in stored_text.items():
            f.write(f'=========={url}===========\n')
            clean_h1 = []
            f.write(f'<<h1>>\n')
            for h1 in texts['h1']:
                clean_h1 += sent_tokenize(h1)
            for sentence in clean_h1:
                f.write(clean_sentence(sentence))
                f.write('\n')
            f.write(f'<<p>>\n')
            clean_p = []
            for p in texts['p']:
                clean_p += sent_tokenize(p)
            for sentence in clean_p:
                f.write(clean_sentence(sentence))
                f.write('\n')
            f.write('\n\n')
            print(f"Finishing with url: {url}")
    print("Document ready")


def main(input_text,output,num_pages=10):
    print("Getting the google page...")
    gg_url = get_google_search_page(input_text)
    print("Get the first page results....")
    top_search = get_top_results(gg_url) 
    print(f"Duplicate and filter to {num_pages} pages...")
    unique_top_search = filter_duplicate(top_search,num_pages)
    print("Extracting the h1 and p elements from these pages...")
    stored_text = extract_text_elems_from_urls(unique_top_search)
    print("Writing to file")
    write_text_dict_to_file(stored_text,output)
    
main('sian','./inference/sian.txt',num_pages = 100)

Getting the google page...
Get the first page results....
Duplicate and filter to 100 pages...
Extracting the h1 and p elements from these pages...
Writing to file
Finishing with url: https://www.bbc.com/news/magazine-33809914
Finishing with url: https://www.lamborghini.com/en-en/models/limited-series/sian-fkp-37
Finishing with url: https://www.lego.com/en-sg/product/lamborghini-sian-fkp-37-42115
Finishing with url: https://www.quora.com/What-does-Sian-Singlish-mean-in-English
Finishing with url: https://www.nortonrosefulbright.com/en-sg/people/123129
Finishing with url: https://www.autoexpress.co.uk/lamborghini/sian
Document ready


In [94]:
#cleaning the crawled text for inference
# cleaning
inference_target = 'sian'
inference_file = f"./inference/{inference_target}.txt"
inference_output_file = f"./inference/clean_{inference_target}.txt"
with open(inference_output_file,'w') as o:
    with open(inference_file,'r',encoding='utf-8') as f:
        lines = f.readlines()
        for line in tqdm(lines):
            result = extract(line,inference_target,10,"<pad>",True)
            if result == None:
                continue
            else:
                print(result)
                o.write(f'{inference_target}:')
                o.write(''.join([i+' ' for i in result] ))
                o.write('\n')
                line_written += 1
                if line_written % 10000 == 0:
                    print(f"Written {line_written} into file {inference_target}")

100%|█████████████████████████████████████████████████████████████████████████████| 334/334 [00:00<00:00, 27829.27it/s]

['among', 'students', 'include', 'mugging', 'means', 'studying', 'hard', 'siao', 'meaning', 'extreme', 'meaning', 'tired', 'boring', 'vanessa', 'kin', 'singapore', '<pad>', '<pad>', '<pad>', '<pad>']
['<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', 'one', 'word', 'singlish', 'used', 'many', 'contexts', 'accurately', 'expresses', 'emotions', 'like']
['<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', 'singlish', 'everyday', 'go', 'work', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
['<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', 'singlish', 'plane', 'delay', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
['<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', 'new', 'lamborghini', 'review', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
['<pad>', 'beneath', 'dramatic', 'new', 'carbonfibre',

In [95]:
with open(inference_output_file,'r') as f:
    batch = f.readlines()
    
sample_batch_tensor = []
target_batch_tensor = []
for sentence in batch:
    y,x = sentence.split(':')
    x = re.sub('[\n\r\ ]+',' ',x).strip()
    sample_tensor = [get_glove_vec(word,W_norm,vocab) for word in x.split(' ')]
    target_batch_tensor.append(get_glove_vec(y,W_norm,vocab))
    sample_batch_tensor.append(sample_tensor)
    
sample_batch_tensor = torch.Tensor(np.array(sample_batch_tensor))
target_batch_tensor = np.array(target_batch_tensor)

sample_output = model(sample_batch_tensor)

output = torch.mean(sample_output,0)   #sum across embeddings
infer_vec_output = output.detach().numpy().reshape((1,100))

print(f"Inference batch: {inference_target}\n\n")

__distance(W_norm,vocab,ivocab,infer_vec_output)


print(f"\n\n\t\tCosim score: {cosim(infer_vec_output,target_label)}")

Inference batch: sian


(400000,)

                               Word       Unnormalized Cosine distance

---------------------------------------------------------

0                              tired		0.531570
1                             scared		0.414386
2                              weary		0.412458
3                                 'm		0.400601
4                         frustrated		0.399057
5                          exhausted		0.392542
6                             afraid		0.389355
7                               feel		0.388964
8                              bored		0.386259
9                           fatigued		0.386248


		Cosim score: [0.15159323]


## <a class="anchor" id="s6">Section 6: Incorporating synthesis vector back into GloVe</a>

In [194]:
## section 6: writing the vectors back 


#choose an output vec that performs well 
output_vec = infer_vec_output
embeddings = {inference_target:output_vec}  #word: embedding

original_glove = './glove_6B/glove.6B.100d.txt'
modified_glove = './glove_6B/modded_glove.6B.100d.txt'

#the word might have existed in the corpus, so we will play safe
existing = []
with open(modified_glove,'w',encoding='utf-8') as f:
    with open(original_glove,'r',encoding='utf-8') as i:
        data = i.readlines()
        for line in data:
            word = line.rstrip().split(' ')[0]
            if word in embeddings:  
                f.write(f"{word} ")  
                f.write(' '.join([f'{i:.5f}' for i in embeddings[word].flatten().tolist()]))
                f.write('\n')
                existing.append(word)
            else:
                f.write(line)
        non_existing = [i for i in embeddings.keys() if i not in existing]
        for new_word in non_existing:
            f.write(f"{word} ")  
            f.write(' '.join([f'{i:.5f}' for i in embeddings[word].flatten().tolist()]))
            f.write('\n')

#checking if it has been incorporated
with open(modified_glove,'r',encoding='utf-8') as f:
    temp = f.readlines()
print(len(temp))
(m_W_norm, mod_vocab, m_ivocab) = generate(modified_glove)
print(len(mod_vocab))
print(existing,non_existing)

400000
400000
['sian'] []
